In [ ]:
# Install CUDA toolkit
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/10.1/Prod/local_installers/cuda-repo-ubuntu1804-10-1-local-10.1.168-418.67_1.0-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-10-1-local-10.1.168-418.67_1.0-1_amd64.deb
!apt-key add /var/cuda-repo-<version>/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-1


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvidia-driver-550-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-docker2' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-doc' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-server-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-headless-390' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-gcc' for glob 'nvidia*'
Note, selecting 'nvidia-headless-418' for glob 'nvidia*'
Note, selecting 'nvidia-headless-430' for glob 'nvidia*'
Note, selecting 'nvidia-headless-435' for glob 'nvidia*'
Note, selecting 'nvidia-headless-440' for glob 'nvidia*'
Note, selecting 'nvidia-headless-450' for glob 'nvidia*'
Note, selecting 'nvidia-headless-455' for glob 'nvidia*'
Note, selecting 'nvidia-headless-460' for glob 'nvidia*'
Note, se

In [ ]:
# Python code in Google Colab
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for pycuda (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pycuda
Failed to build pycuda
ERROR: Could not build wheels for pycuda, which is required to install pyproject.toml-based projects


In [ ]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

ModuleNotFoundError: No module named 'pycuda'

In [ ]:
# Load CUDA kernel code from file
with open('reduction_kernel.cu', 'r') as file:
    reduction_kernel = file.read()

# Compile the CUDA kernel
mod = SourceModule(reduction_kernel)

Saving reduction_kernel.cu to reduction_kernel.cu


In [ ]:
# Get the functions from the compiled module
min_reduction = mod.get_function("min_reduction")
max_reduction = mod.get_function("max_reduction")
sum_reduction = mod.get_function("sum_reduction")
avg_reduction = mod.get_function("avg_reduction")

/bin/bash: line 1: nvcc: command not found


In [ ]:
# Example input data
input_data = np.array([0, 1, 2, 3, 4, 5], dtype=np.float32)  # Example input data (replace with your data)
data_size = input_data.size

# Allocate GPU memory for input and output data
input_gpu = cuda.mem_alloc(input_data.nbytes)
output_gpu = cuda.mem_alloc(input_data.nbytes)

# Copy input data to GPU memory
cuda.memcpy_htod(input_gpu, input_data)

# Define block and grid dimensions
block_dim = (256, 1, 1)
grid_dim = ((data_size - 1) // 256 + 1, 1)

# Execute the CUDA kernels for min, max, sum, and average
min_reduction(input_gpu, output_gpu, np.int32(data_size), block=block_dim, grid=grid_dim)
max_reduction(input_gpu, output_gpu, np.int32(data_size), block=block_dim, grid=grid_dim)
sum_reduction(input_gpu, output_gpu, np.int32(data_size), block=block_dim, grid=grid_dim)
avg_reduction(input_gpu, output_gpu, np.int32(data_size), block=block_dim, grid=grid_dim)

# Copy the results back to the host
min_result = np.empty_like(input_data)
max_result = np.empty_like(input_data)
sum_result = np.empty_like(input_data)
avg_result = np.empty_like(input_data)

cuda.memcpy_dtoh(min_result, output_gpu)
cuda.memcpy_dtoh(max_result, output_gpu)
cuda.memcpy_dtoh(sum_result, output_gpu)
cuda.memcpy_dtoh(avg_result, output_gpu)

print("Input Data:", input_data)
print("Minimum Value:", min_result)
print("Maximum Value:", max_result)
print("Sum:", sum_result)
print("Average:", avg_result)

/bin/bash: line 1: ./reduction_kernel.cu: Permission denied
